In [19]:
import cv2 as cv
import numpy as np
from tqdm import tqdm

In [20]:
# utils
def timestamp_to_framestamp(t, fps):
    return np.round(np.multiply(t, fps)).astype(int)

def framestamp_to_timestamp(f, fps):
    return np.divide(f, fps)

# Crop video

In [18]:
def crop_video(input_video_path, output_video_path, x, y, w, h):
    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    # Define the codec and create VideoWriter object
    out = cv.VideoWriter(output_video_path, fourcc, fps, (w, h))

    # Process each frame
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # Crop the frame
        cropped_frame = frame[y:y+h, x:x+w]

        # Write the cropped frame to the output video
        out.write(cropped_frame)

    # Release everything when job is finished
    cap.release()
    out.release()

# Extract Scoring Timestamps

In [23]:
def get_score_framestamps(input_video_path, basket_bound_x=(115, 160), basket_bound_y=(110, 115), cooldown_seconds=1):
    """
    Records a frame as a score if the ball enters the specified basket bounding box.
    The score bounding box should be aligned to the top of the basket.

    params:
        input_video_path: 
        output_video_path: 
        basket_bound_x: tuple
        basket_bound_y: tuple
        cooldown_seconds: length of time before the next detection is allowed
    """
    line_colour = (227, 73, 121)

    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    # removes the background using KNN algorithm, which is good if small parts of a complex background change frequently
    # https://www.reddit.com/r/opencv/comments/yrbl07/question_how_does_knn_background_subtractor_work/
    # https://en.wikipedia.org/wiki/Kernel_density_estimation 
    # tl;dr learns a background model by progressively applying each frame to the model
    fgbg = cv.createBackgroundSubtractorKNN()

    prev_seconds = -1
    framestamps = []
    # Process each frame
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # remove background
        mask = fgbg.apply(frame)
        # median blur to denoise https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
        mask = cv.medianBlur(mask, 5)

        if not np.any(mask):
            continue

        # calc ball's center of mass using pixels
        com = (np.mean(np.argwhere(mask), axis=0)).astype(int)
        cv.circle(frame, com[::-1], 3, line_colour, 2)

        cv.line(frame, (basket_bound_x[0], basket_bound_y[0]), (basket_bound_x[1], basket_bound_y[0]), line_colour, 2)
        # if com crosses the line, print timestamp
        if basket_bound_y[0] < com[0] < basket_bound_y[1] and basket_bound_x[0] < com[1] < basket_bound_x[1]:
            current_seconds = i / fps
            if not framestamps or current_seconds >= prev_seconds + cooldown_seconds:
                print(f"basket made at: {current_seconds//60:.0f}m{current_seconds%60:.0f}s")
                prev_seconds = current_seconds
                framestamps.append(i)

    # Release everything when job is finished
    cap.release()

    print("Found ", len(framestamps), " scored baskets.")
    return framestamps

# Highlight segmentation

In [12]:
from pathlib import Path

def segment_video(input_video_path, score_framestamps, before_score_seconds=3, after_score_seconds=1, split_segments=False):
    """
    Params:
        input_video_path: path to video to segment; output highlight reel
        score_framestamps: list of frames where a basket was scored
        before_score_seconds: buffer length before the shot
        highlight_len_after_frames: buffer length after the shot
        split_segments: whether each segment should be its own video
    """
    # Open the input video
    cap = cv.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    original_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    highlight_frames = before_score_seconds * fps
    highlight_num = 0
    out = cv.VideoWriter(f"{Path(input_video_path).stem}_highlight.mp4", fourcc, fps, (original_width, original_height))
    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        if highlight_num == len(score_framestamps):
            break

        start_frame = max(0, score_framestamps[highlight_num] - highlight_frames) # no negative frame numbers
        if i > start_frame:
            # if reached end of highlight, move on to next highlight
            if i > start_frame + highlight_frames + after_score_seconds*fps:
                print(f"highlight {highlight_num} complete")
                highlight_num += 1
                if split_segments:
                    out.release()
                    out = cv.VideoWriter(f"{Path(input_video_path).stem}_highlight_{highlight_num}.mp4", fourcc, fps, (original_width, original_height))
            else:
                out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()

# snip snip!!

[Website](https://yt5s.biz/watch?v=ACR3RUiV6Rw) for free youtube audio download

[Clideo](https://clideo.com/editor/add-audio-to-video) for adding audio to highlight reel (TODO do this in python so no watermarks :p)

In [21]:
# choose your file!
input_video_path = "C0013.MP4"
cropped_video_path = "output_cropped.mp4"

In [ ]:
x, y, w, h = 750, 280, 250, 150  # Specify the x, y, width, and height of basket
# TODO bound basket automatically with polygon detection?
crop_video(input_video_path, cropped_video_path, x, y, w, h)

In [24]:
score_framestamps = get_score_framestamps(cropped_video_path, (115,160), (105, 115))


 12%|█▏        | 2250/19512 [00:02<00:21, 814.80it/s]

basket made at: 0m43s


 15%|█▌        | 2951/19512 [00:03<00:21, 785.91it/s]

basket made at: 0m56s


 18%|█▊        | 3533/19512 [00:04<00:19, 808.90it/s]

...

basket made at: 5m43s


 96%|█████████▌| 18638/19512 [00:23<00:01, 694.64it/s]

basket made at: 6m11s


100%|██████████| 19512/19512 [00:24<00:00, 782.71it/s]


Found  11  scored baskets.


In [15]:
segment_video(input_video_path, score_framestamps, before_score_seconds=5, after_score_seconds=2)

 12%|█▏        | 2277/19512 [00:40<08:00, 35.85it/s]

highlight 0 complete


 15%|█▌        | 2928/19512 [01:02<07:45, 35.61it/s]

highlight 1 complete


 18%|█▊        | 3558/19512 [01:23<08:16, 32.15it/s]

...

highlight 8 complete


 88%|████████▊ | 17263/19512 [14:06<00:55, 40.64it/s]

highlight 9 complete


 96%|█████████▌| 18655/19512 [14:37<00:40, 21.26it/s]

highlight 10 complete
